In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
import pandas as pd 

from scipy import optimize
from scipy.optimize import curve_fit
from scipy import integrate
from scipy.integrate import odeint

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html

from dash.dependencies import Input, Output, State

import plotly
import plotly.graph_objects as go


import os

print(os.getcwd())


sns.set(style="darkgrid")

mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 500)

C:\WINDOWS\system32\applied_data_science_-_covid-19


In [2]:
data_c = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
data_c = data_c.drop(["Province/State", "Lat","Long"], axis =1)
data_c = data_c.groupby(["Country/Region"]).sum()
data_r = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
data_r = data_r.drop(["Province/State", "Lat","Long"], axis =1)
data_r = data_r.groupby(["Country/Region"]).sum()
data_d = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
data_d = data_d.drop(["Province/State", "Lat","Long"], axis =1)
data_d = data_d.groupby(["Country/Region"]).sum()

In [3]:
def sir_simulations( data_c, data_r, data_d, dates):
    duration = 25
    
    data_c = data_c[(len(data_c)-duration):]
    data_r = data_r[(len(data_r)-duration):]
    data_d = data_d[(len(data_d)-duration):]
    dates = dates[len(dates)-duration: ]
    N = 1000000
    I0 = data_c[0]
    R0 = data_r[0]
    S0 = N- R0- I0
    
    def SIR(y, t, beta, gama):
        S = y[0]
        I = y[1]
        R = y[2]
        return -beta*S*I/N, (beta*S*I)/N- (gama*I), gama*I
    
    def fit_odeint(t, beta, gama):
        return odeint(SIR,(S0,I0,R0), t, args = (beta,gama))[:,1]
    t = np.arange(len(data_c))
    params, cerr = curve_fit(fit_odeint,t, data_c)
    beta,gama = params
    prediction = list(fit_odeint(t,beta,gama))
    
    
    fig = go.Figure()
    fig.add_traces(go.Scatter(x= dates, y= prediction,
                              mode='lines',
                              name='simulated'))
    fig.add_bar(x= dates, y= data_c, name="Actual")
    fig.update_layout(height = 800,
                      title = {
                          'text':"SIR simulation",
                          'x' : 0.5},
                      xaxis_title='Date',
                      yaxis_title='Infections')
    return fig

In [4]:
#fig = go.Figure()
#fig.plot(confirmed_cases, recovered_cases, dates)

In [5]:
fig = go.Figure()

In [6]:
last_date = data_d.T.index[-1]
data_table = pd.DataFrame([data_c[last_date], data_r[last_date], data_d[last_date]]).T
data_table.columns = ['confirmed', 'recovered', 'deaths']
data_table['active'] = data_table.confirmed - data_table.recovered - data_table.deaths
data_table['percent_deaths'] = np.around(data_table.deaths / data_table.confirmed, 2)
data_table['percent_recoveries'] = np.around(data_table.recovered / data_table.confirmed, 2)
data_table = data_table.sort_values(by = 'confirmed', ascending = False)
data_table.reset_index(level=0, inplace=True)

In [7]:
app= dash.Dash(__name__, suppress_callback_exceptions=True)
server=app.server
#df = data_table()
#data_c, data_r, data_d = data()
str_dates = data_c.columns
start_date = str_dates[0].split('/')
end_date = str_dates[-1].split('/')
if len(end_date[0]) == 1:
    dates = np.arange(np.datetime64(f'20{start_date[2]}-0{start_date[0]}-{start_date[1]}'), np.datetime64(f'20{end_date[2]}-0{end_date[0]}-{int(end_date[1])+1}'))
else:
    dates = np.arange(np.datetime64(f'20{start_date[2]}-0{start_date[0]}-{start_date[1]}'), np.datetime64(f'20{end_date[2]}-{end_date[0]}-{int(end_date[1])+1}'))




app.layout = html.Div([
    
    html.Label('SIR_modelling'),
    
    
    dcc.Dropdown(
        id="country_list",
        options=[ 
            {'label':country_name, 'value': country_name} for country_name in list(data_c.index)
        ],
        value= 'Germany',
        multi = False
    ),
    
    dcc.Graph(figure=fig, id='sir_simulations')
    ])


 

In [8]:
@app.callback(
    Output ('sir_simulations' , 'figure'),
    [Input('country_list', 'value')])
def country_data(country):
    
    confirmed_cases = list(data_c.loc[f'{country}'])
    recovered_cases = list(data_r.loc[f'{country}'])
    death_cases = list(data_d.loc[f'{country}'])
    #dates = dates[len(dates)-duration: ]
    
    sir_figure = sir_simulations(confirmed_cases, recovered_cases,death_cases, dates)
    
    return sir_figure
    

In [9]:
app.run_server(debug=True, use_reloader= False)

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.



 * Tip: There are .env or .flaskenv files present. Do "pip install python-dotenv" to use them.


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
